# mmult unit test

In [1]:
import numpy as np
import cffi

from pynq import Overlay

# load Base Overlay
Overlay("/home/xilinx/pynq/bitstream/base.bit").download()

import sys
sys.path.append("..")

from pynq_chainer import functions as F_
from pynq_chainer import links as L_
from pynq_chainer import overlays
from pynq_chainer import utils

pcsim is not run on pynq


In [19]:
from pynq.drivers import xlnk
mmu = xlnk.xlnk()
mmu.xlnk_reset()

In [20]:
mmult = overlays.Mmult()
ffi = cffi.FFI()

In [21]:
def debug_cdata(name, cdata, show=False):
    if not show:
        return
    print(name)
    for i in range(5):
        print(cdata[i])

In [24]:
def test(debug=False):
    ffi = cffi.FFI()
    
    w_size = (1024, 32)
    x_size = (512, 32)
    w_size = (1, 784)
    x_size = (32, 784)

    w = np.random.uniform(-1, 1, w_size).astype(np.float32)
    x = np.random.uniform(-1, 1, x_size).astype(np.float32)

    x_nrows, x_ncols = x.shape
    w_nrows, w_ncols = w.shape
    y, y_cdata = utils.malloc_cma_ndarray((w_nrows, x_nrows))
    
    x, x_cdata = utils.copy_cma_ndarray(x)
    w, w_cdata = utils.copy_cma_ndarray(w)
    
    if debug:
        print('x', x_cdata)
        print('w', w_cdata)
        print('y', y_cdata)
    debug_cdata("x", x_cdata)
    debug_cdata("w", w_cdata)
    debug_cdata("y", y_cdata)

    mmult(x_cdata, w_cdata, y_cdata, x_nrows, w_nrows, x_ncols)
    
    debug_cdata("y", y_cdata)
    y_ = x.dot(w.T)
    y = y.T
    
    debug_cdata("y", y_cdata)

    if debug:
        print("Actual(C):")
        print(y)
        print("Expected(NumPy):")
        print(y_)
    
    if np.allclose(y, y_, rtol=1e-04, atol=1e-04):
        print("OK")
    else:
        print("NG")
        
    mmu.cma_free(x_cdata)
    mmu.cma_free(w_cdata)
    mmu.cma_free(y_cdata)

In [25]:
for i in range(100):
    test(debug=False)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


In [31]:
w_size = (1, 784)
x_size = (1000, 784)

w = np.random.uniform(-1, 1, w_size).astype(np.float32)
x = np.random.uniform(-1, 1, x_size).astype(np.float32)

x_nrows, x_ncols = x.shape
w_nrows, w_ncols = w.shape
y, y_cdata = utils.malloc_cma_ndarray((w_nrows, x_nrows))

x, x_cdata = utils.copy_cma_ndarray(x)
w, w_cdata = utils.copy_cma_ndarray(w)

In [32]:
%timeit -n 100 -o x.dot(w.T)

100 loops, best of 3: 11.4 ms per loop


<TimeitResult : 100 loops, best of 3: 11.4 ms per loop>

In [33]:
%timeit -n 100 -o mmult(x_cdata, w_cdata, y_cdata, x_nrows, w_nrows, x_ncols)

100 loops, best of 3: 94.9 ms per loop


<TimeitResult : 100 loops, best of 3: 94.9 ms per loop>